#### Домашнее задание
* Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) которые бы
  задавались следующим образом:
* x - последовательность цифр
* y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10
* Задача научить модель предсказывать y(i) по x(i)
* Опробовать RNN, LSTM, GRU длинной 10, 25, 50 , 100

In [1]:
import torch
from torch import nn
import re
import random
import tqdm
import time
import numpy as np
from time import sleep

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
def num(num):
    x = torch.randint(0, 9, (num, ))
    y = torch.zeros(x.shape[0]).int()
    for i, j in enumerate(x):
        if i == 0:
            y[i] = x[i]
        else:
            y[i] = x[i] + x[0]
            if y[i] >= 10:
                y[i] -= 10
    x = x.reshape(1, num)
    y = y.reshape(1, num)            
    return x, y  


In [5]:
num(5)

(tensor([[1, 3, 4, 6, 6]]), tensor([[1, 4, 5, 7, 7]], dtype=torch.int32))

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self,  rnnClass, dictionary_size, embedding_size = 10, num_hiddens = 128, num_classes = 10):
        super().__init__()
        self.rnnClass = rnnClass
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        if self.rnnClass == nn.LSTM:
            predictions = self.output(state[0][0])
        else:
            predictions = self.output(state[0])

        return predictions


In [10]:
def sample(preds):
    soft = nn.LogSoftmax(dim=1)    
    softmaxed = soft(preds)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax(1)

def generate_num(i):
    x, y = num(i)
    for j in range(i):

        preds = model((x[0][j]).reshape(1, 1))
        
    return sample(preds), y[0][-1]

In [12]:
rnn = nn.RNN
gru = nn.GRU
lstm = nn.LSTM
nets = {'RNN': rnn, 'GRU': gru, 'LSTM': lstm}
d = [10, 25 , 50, 100]

In [16]:
# for k, v in net.items():
#     model = NeuralNetwork(v, 10, 10, 128, 10)

def net_model(k, v, i, model):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    print("_" * 20)
    print("модель:", k, "Длина:", i)
    
    rnn_dict = {}
    x, y = num(i)
    emb = nn.Embedding(10, 10)
    for ep in range(10):
        for h in range(i):
                   

            n = x[0][h].reshape(1, 1)
            j = y[0][h].view(1,)

            start = time.time()
            train_loss = 0.
            train_passed = 0

            model.train() 

            optimizer.zero_grad()
            answers = model(n)  


            loss = criterion(answers, emb(j))
            train_loss += loss.item()


            loss.backward(retain_graph = True)    
            optimizer.step()
            train_passed += 1    

        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        model.eval()
        gen = generate_num(i)
        l = np.where((gen[0].squeeze() - gen[1].squeeze())==0)    
        print(f' "Точность"', abs((10-abs(gen[0].squeeze() - gen[1].squeeze())).item())*10, '%')
        print((gen[0].squeeze(), gen[1].squeeze()))
        if (gen[0].squeeze() - gen[1].squeeze()).item() == 0:
            return

In [17]:
for k, v in nets.items():
    for i in d:
        model = NeuralNetwork(v, i)
        net_model(k, v, i, model)

____________________
модель: RNN Длина: 10
Epoch 0. Time: 0.001, Train loss: -0.777
 "Точность" 60 %
(tensor(7), tensor(3, dtype=torch.int32))
Epoch 1. Time: 0.001, Train loss: -1.769
 "Точность" 70 %
(tensor(3), tensor(0, dtype=torch.int32))
Epoch 2. Time: 0.001, Train loss: -2.882
 "Точность" 100 %
(tensor(5), tensor(5, dtype=torch.int32))
____________________
модель: RNN Длина: 25
Epoch 0. Time: 0.002, Train loss: -11.518
 "Точность" 100 %
(tensor(0), tensor(0, dtype=torch.int32))
____________________
модель: RNN Длина: 50
Epoch 0. Time: 0.001, Train loss: -0.052
 "Точность" 80 %
(tensor(1), tensor(3, dtype=torch.int32))
Epoch 1. Time: 0.001, Train loss: -0.995
 "Точность" 60 %
(tensor(4), tensor(8, dtype=torch.int32))
Epoch 2. Time: 0.001, Train loss: -2.210
 "Точность" 10 %
(tensor(0), tensor(9, dtype=torch.int32))
Epoch 3. Time: 0.001, Train loss: -4.372
 "Точность" 80 %
(tensor(6), tensor(8, dtype=torch.int32))
Epoch 4. Time: 0.001, Train loss: -7.774
 "Точность" 100 %
(tensor(0